<a href="https://colab.research.google.com/github/Tahm24/ModelB/blob/main/MainModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Spacy Model Creation - Key word extractor (Custom Dataset)**

In [ ]:
# Install libraries
!pip install -q spacy==3.8.5
!python -m spacy download en_core_web_sm

# Imports
import spacy
import json
import random
from pathlib import Path
from spacy.tokens import DocBin
from sklearn.model_selection import train_test_split
from google.colab import files

# Uploaded .jsonl file
uploaded = files.upload()

# Uploaded filename manually
jsonl_file = list(uploaded.keys())[0]

# Load custom NER data
examples = []
with open(jsonl_file, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        text = data['text']
        entities = data['entities']
        examples.append((text, {"entities": [(start, end, label) for start, end, label in entities]}))

print(f" Loaded {len(examples)} training examples.")

# Split into train/dev
train_data, dev_data = train_test_split(examples, test_size=0.1, random_state=42)

# Helper to convert into spaCy format
def create_docbin(data, nlp):
    db = DocBin()
    for text, annot in data:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot['entities']:
            span = doc.char_span(start, end, label=label)
            if span:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return db

# Create output folders
Path("data").mkdir(parents=True, exist_ok=True)
nlp_blank = spacy.blank("en")  # Blank English model

create_docbin(train_data, nlp_blank).to_disk("data/train.spacy")
create_docbin(dev_data, nlp_blank).to_disk("data/dev.spacy")

# config file optimised for CPU
!python -m spacy init config data/config.cfg --lang en --pipeline ner --optimize efficiency --force

# Train model
!python -m spacy train data/config.cfg --output model --paths.train data/train.spacy --paths.dev data/dev.spacy --gpu-id -1

print("\n Model Training Complete!")

# Load and test trained model via data outside training / testval
import spacy

nlp_trained = spacy.load("model/model-best")

# Test text
test_text = """
John Doe — Backend Developer skilled in Python, Flask, and PostgreSQL.
Worked at DevSolutions Ltd from 2020-2024. MSc Computer Science, University of Manchester.
"""

doc = nlp_trained(test_text)

print("\nEntities Found:\n")
for ent in doc.ents:
    print(f"{ent.text} ({ent.label_})")

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Saving software_cv_ner2.json to software_cv_ner2 (1).json
✅ Loaded 17 training examples.
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
data/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
ℹ Saving to output directory: model
ℹ Using CPU
ℹ To swi

In [ ]:
import re

# CV text
cv_text = """
James Carter
Email: james.carter@example.com
Phone: +44 1234 567890
LinkedIn: linkedin.com/in/jamescarter
GitHub: github.com/jcarter
Location: London, UK

Professional Summary
Software Engineer with 4+ years of experience in designing, developing, and maintaining scalable software solutions. Adept in Java, Python, and JavaScript frameworks, with strong skills in Agile methodologies, teamwork, and problem-solving.

Technical Skills
Languages: Java, Python, JavaScript, C++
Frameworks: React, Angular, Node.js, Django, Spring Boot
Tools & Platforms: Git, Docker, Kubernetes, AWS, Azure, Jenkins
Databases: MySQL, PostgreSQL, MongoDB

Professional Experience
Software Engineer | TechSolutions Ltd., London, UK
March 2021 – Present
Responsibilities:
- Developed and maintained web applications using React, Node.js, and MongoDB.
- Improved application performance by 25% through code optimisation and refactoring.
- Led integration of RESTful APIs, enhancing application scalability.
- Collaborated with cross-functional teams using Agile methodologies.

Junior Software Developer | Innovatech Inc., London, UK
January 2019 – February 2021
Responsibilities:
- Assisted in developing backend systems with Java (Spring Boot) and Python (Django).
- Contributed to database design and management, optimising query performance by 15%.
- Participated in code reviews, debugging sessions, and maintained coding standards.

Education
BSc Computer Science | University College London, UK | Graduated: 2018

Certifications
- AWS Certified Solutions Architect – Associate (2022)
- Oracle Certified Java Programmer (2021)

Projects
Inventory Management System (React, Node.js, MongoDB) – Developed a full-stack web application to manage warehouse inventory.
Chat Application (Java, Spring Boot, WebSocket) – Created a real-time chat application with secure authentication.

Languages
- English (Native)
- French (Intermediate)
"""





# Initialize the extracted fields
extracted_info = {}

# Extract name (first non-empty line)
lines = [line.strip() for line in cv_text.strip().split('\n') if line.strip()]
extracted_info['Name'] = lines[0]

# Extract email
email_match = re.search(r'[\w\.-]+@[\w\.-]+', cv_text)
extracted_info['Email'] = email_match.group(0) if email_match else None

# Extract phone number
phone_match = re.search(r'\+?\d[\d\s\-]{7,}\d', cv_text)
extracted_info['Phone'] = phone_match.group(0) if phone_match else None

# Extract LinkedIn URL
linkedin_match = re.search(r'linkedin\.com\/[^\s]+', cv_text)
extracted_info['LinkedIn'] = linkedin_match.group(0) if linkedin_match else None

# Extract Professional Title (first line under "Professional Summary")
prof_summary_match = re.search(r'Professional Summary\n([^\n]+)', cv_text)
if prof_summary_match:
    first_line_summary = prof_summary_match.group(1)
    # Assume the profession is the first few words before "with" or comma
    profession_match = re.match(r'(.+?)(?: with|,)', first_line_summary)
    extracted_info['Profession Title'] = profession_match.group(1) if profession_match else first_line_summary

# Extract Years of Experience
years_match = re.search(r'(\d+)\+?\s*years? of experience', cv_text, re.IGNORECASE)
extracted_info['Years of Experience'] = years_match.group(1) if years_match else None

# Print all extracted fields
for field, value in extracted_info.items():
    print(f"{field}: {value}")


Name: James Carter
Email: james.carter@example.com
Phone: +44 1234 567890
LinkedIn: linkedin.com/in/jamescarter
Profession Title: Software Engineer
Years of Experience: 4


# **Pre-Trained Model using all-mpnet-V2**

In [ ]:
!pip install -q sentence-transformers python-docx

import torch
from sentence_transformers import SentenceTransformer, util
import logging
from transformers.utils import logging as hf_logging
import numpy as np

# Logging
logging.basicConfig(level=logging.INFO)
hf_logging.set_verbosity_info()

class CVMatcher:
    def __init__(self, model_name='all-mpnet-base-v2', device=None):
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        logging.info(f"Loading model: {model_name} on {self.device}")
        self.model = SentenceTransformer(model_name, device=self.device)
        logging.info("Model loaded successfully.")

    def segment_text(self, text, max_length=256):
        segments = []
        current = []
        count = 0
        for line in text.split('\n'):
            line = line.strip()
            if not line:
                continue
            current.append(line)
            count += len(line.split())
            if count >= max_length:
                segments.append(' '.join(current))
                current = []
                count = 0
        if current:
            segments.append(' '.join(current))
        return segments

    def compute_similarity(self, cv_text, job_text) -> float:
        cv_chunks = self.segment_text(cv_text)
        job_chunks = self.segment_text(job_text)

        print(f"Comparing {len(cv_chunks)} CV segments to {len(job_chunks)} job segments...")

        cv_embeddings = self.model.encode(cv_chunks, convert_to_tensor=True)
        job_embeddings = self.model.encode(job_chunks, convert_to_tensor=True)

        sim_matrix = util.cos_sim(cv_embeddings, job_embeddings).cpu().numpy()
        max_sim = np.max(sim_matrix)

        print(f"Max segment similarity: {max_sim * 100:.2f}%")
        return max_sim * 100

    def rank_jobs(self, cv: str, jobs: dict, top_k: int = None, verbose=True):
        print("Ranking jobs based on similarity to the CV\n")
        results = []
        for title, desc in jobs.items():
            print(f"Evaluating job: {title}")
            score = self.compute_similarity(cv, desc)
            results.append((title, score))
        ranked = sorted(results, key=lambda x: x[1], reverse=True)
        if verbose:
            print("\nFinal Job Matching Results:")
            for title, score in ranked[:top_k or len(ranked)]:
                print(f" - {title:40s}: {score:.2f}%")
        return ranked


In [ ]:
from google.colab import files
import docx
import os

# Upload CV
uploaded = files.upload()
docx_filename = next(iter(uploaded))

# Read CV text
def read_docx_text(path):
    doc = docx.Document(path)
    return "\n".join([para.text for para in doc.paragraphs if para.text.strip()])

cv_text = read_docx_text(docx_filename)

# jobs test
jobs = {
    "Software Engineer (React/AWS)": """
        We are seeking a full-stack software engineer to develop cloud-native applications using React, Node.js,
        and AWS. Experience with containerisation tools like Docker and CI/CD pipelines using Jenkins is required.
        Must be a strong team player and comfortable working in Agile environments.
    """,
    "Data Scientist": """
        We are looking for a Data Scientist with solid experience in Python, deep learning, and ML frameworks like
        PyTorch and TensorFlow. Knowledge in NLP, pandas, scikit-learn, and AWS is highly desirable. Candidates
        should be able to build models, run experiments, and deliver production-grade systems.
    """,
    "Random Text": "I have quick react speed on cars"
}

# Run matcher
matcher = CVMatcher()
matcher.rank_jobs(cv=cv_text, jobs=jobs)

# Cleanup uploaded file
os.remove(docx_filename)


Saving Machine_Learning_CV.docx to Machine_Learning_CV.docx


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--sentence-transformers--all-mpnet-base-v2/snapshots/12e86a3c702fc3c50205a8db88f0ec7c0b6b94a0/config.json
Model config MPNetConfig {
  "architectures": [
    "MPNetForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "mpnet",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "relative_attention_num_buckets": 32,
  "transformers_version": "4.51.3",
  "vocab_size": 30527
}

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--sentence-transformers--all-mpnet-base-v2/snapshots/12e86a3c702fc3c50205a8db88f0ec7c0b6b94a0/model.safetensors
All model checkpoint weights were used when initial

Ranking jobs based on similarity to the CV

Evaluating job: Software Engineer (React/AWS)
Comparing 2 CV segments to 1 job segments...
Max segment similarity: 48.83%
Evaluating job: Data Scientist
Comparing 2 CV segments to 1 job segments...
Max segment similarity: 70.55%
Evaluating job: Random Text
Comparing 2 CV segments to 1 job segments...
Max segment similarity: 18.17%

Final Job Matching Results:
 - Data Scientist                          : 70.55%
 - Software Engineer (React/AWS)           : 48.83%
 - Random Text                             : 18.17%
